In [15]:
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_ollama import ChatOllama

import aiohttp
import json
from IPython.display import display



### Get JWT token

In [ ]:
base_url = "http://YOUR_IP_ADDRESS:8000" # Asset Management API

async def get_user_token(username: str, password: str) -> str:
    """Authenticate and get a jwt token"""
    async with aiohttp.ClientSession() as session:
        async with session.post(
            f"{base_url}/token", 
            data={
                "username": username,
                "password": password
            }
        ) as resp:
            result = await resp.json()

    return result["access_token"]


### Use the tool directly

In [ ]:
# Make sure the files path is available for the mcp server.
# If not, please check the volume mapping in docker-compose.yml
primary_file = "./test/test.txt"
associated_files = ["./test/test.json"]
new_associated_files = ["./test/test.jpg"]

mcp_endpoint = "http://YOUR_IP_ADDRESS:8003/mcp/" # fastmcp
# mcp_endpoint = "http://YOUR_IP_ADDRESS:8004/mcp/" # low-level server

user_token = await get_user_token("admin", "dht888888")
headers = {"Authorization": f"Bearer {user_token}"} # use headers to pass token


# list tools
async with streamablehttp_client(mcp_endpoint, headers=headers) as (read, write, _):
	async with ClientSession(read, write) as session:
		# Initialize the connection
		await session.initialize()
		
		result = await session.list_tools()
		display(result.tools)


# upload file
async with streamablehttp_client(mcp_endpoint, headers=headers) as (read, write, _):
	async with ClientSession(read, write) as session:
		# Initialize the connection
		await session.initialize()

		result = await session.call_tool("upload_file", arguments={"primary_file": primary_file, "associated_files": associated_files})
		print("Upload result:")
		print(result.content[0].text)
		print('*'*100)

		result = json.loads(result.content[0].text)
		asset_path  = result["asset_path"]
		version_id  = result["version_id"]
		primary_filename = result["primary_filename"]


# add associated files
async with streamablehttp_client(mcp_endpoint, headers=headers) as (read, write, _):
	async with ClientSession(read, write) as session:
		# Initialize the connection
		await session.initialize()

		result = await session.call_tool("add_associated_files", arguments={"asset_path": asset_path, "primary_version_id": version_id, "associated_files": new_associated_files})
		print("Add associated files result:")
		print(result.content[0].text)
		print('*'*100)


# list versions
async with streamablehttp_client(mcp_endpoint, headers=headers) as (read, write, _):
	async with ClientSession(read, write) as session:
		# Initialize the connection
		await session.initialize()

		result = await session.call_tool("list_file_versions", arguments={"asset_path": asset_path, "primary_filename": primary_filename})
		print("List versions result:")
		print(result.content[0].text)
		print('*'*100)


# download
async with streamablehttp_client(mcp_endpoint, headers=headers) as (read, write, _):
	async with ClientSession(read, write) as session:
		# Initialize the connection
		await session.initialize()

		result = await session.call_tool("file_download", arguments={"asset_path": asset_path, "version_id": version_id})
		print("Download result:")
		print(result.content[0].text)
		print('*'*100)


# archive
async with streamablehttp_client(mcp_endpoint, headers=headers) as (read, write, _):
	async with ClientSession(read, write) as session:
		# Initialize the connection
		await session.initialize()

		result = await session.call_tool("file_archive", arguments={"asset_path": asset_path, "version_id": version_id})
		print("Archive result:")
		print(result.content[0].text)
		print('*'*100)


# delete
async with streamablehttp_client(mcp_endpoint, headers=headers) as (read, write, _):
	async with ClientSession(read, write) as session:
		# Initialize the connection
		await session.initialize()

		result = await session.call_tool("file_delete", arguments={"asset_path": asset_path, "version_id": version_id})
		print("Delete result:")
		print(result.content[0].text)
		print('*'*100)



[Tool(name='upload_file', title=None, description='\n    Upload a primary file and optional associated files\n\n    Args:\n        primary_file (str): path to the primary file\n        associated_files (List[str], optional): paths to associated files. Defaults to [].\n        archive_ttl (int, optional): archive ttl in days. Defaults to 30.\n        destroy_ttl (int, optional): destroy ttl in days. Defaults to 30.\n\n    Returns:\n        Dict: Information about the uploaded asset, with keys:\n            - asset_path (str): Path of the uploaded asset.\n            - version_id (str): Version ID assigned to the upload.\n            - primary_filename (str): Name of the primary file.\n            - associated_filenames (List[List[str]]): List of associated files and their version IDs, e.g.,\n              [["test.json", "version id"], ["test.jpg", "version id"]].\n            - upload_date (str): Upload timestamp in ISO 8601 format.\n            - archive_date (str): Scheduled archive t

Upload result:
{
  "asset_path": "document/txt/test",
  "version_id": "ef963dd23e2fd71135722fb57499aa44f37063324cbb41c761c2b81b01866030",
  "primary_filename": "test.txt",
  "associated_filenames": [
    [
      "test.json",
      "ace81191e67506f811fa44b65f0766b1a9a6e96f10ef550d1fb20dfdcff31780"
    ],
    [
      "test.jpg",
      "111ec665a811320eafe11d627594540bdf056bdd2bc1b617c0c20687f97a4a14"
    ]
  ],
  "upload_date": "2025-09-22T11:23:41",
  "archive_date": "2025-10-22T11:23:41",
  "destroy_date": "2025-11-21T11:23:41",
  "branch": "admin_space",
  "status": "active"
}
****************************************************************************************************
Add associated files result:
{
  "asset_path": "document/txt/test",
  "version_id": "ef963dd23e2fd71135722fb57499aa44f37063324cbb41c761c2b81b01866030",
  "primary_filename": "test.txt",
  "associated_filenames": [
    [
      "test.json",
      "ace81191e67506f811fa44b65f0766b1a9a6e96f10ef550d1fb20dfdcff31780"
  

### Let the Chat Model Decide How to Call the Tool
Requirements:

An Ollama server running locally or accessible remotely.

In [ ]:
model = "qwen3:14b" # Change this to your ollama model
ollama_host = "http://YOUR_IP_ADDRESS:11434" # Change this to your ollama host

primary_file = "./test/test.txt"
associated_files = ["./test/test.json", "./test/test.jpg"]

mcp_endpoint = "http://YOUR_IP_ADDRESS:8003/mcp/" # fastmcp
# mcp_endpoint = "http://YOUR_IP_ADDRESS:8004/mcp/" # low-level server

user_token = await get_user_token("admin", "dht888888")
headers = {"Authorization": f"Bearer {user_token}"}


async def streaming_token(message, agent, config):
	def print_section(title: str):
		print(f"\n{'='*50}\033[1m {title} \033[0m{'='*50}\n")

	print_section("Human message")
	print(message)

	last_node = None
	async for step, metadata in agent.astream(
		{"messages": [message]}, 
		config, 
		stream_mode="messages"
	):
		node = metadata["langgraph_node"]
		if (text := step.text()):
			if node != last_node:
				print_section(f"{node.capitalize()} message")
				last_node = node
			print(text, end="")


async with streamablehttp_client(mcp_endpoint, headers=headers) as (read, write, _):
	async with ClientSession(read, write) as session:
		# Initialize the connection
		await session.initialize()

		# Get tools
		tools = await load_mcp_tools(session)
		llm = ChatOllama(model=model, base_url=ollama_host, temperature=0, keep_alive=0)
		memory = MemorySaver()

		agent = create_react_agent(llm, tools, checkpointer=memory)
		config = {"configurable": {"thread_id": "abc123"}}

		await streaming_token(
			message=f"""
				幫我上傳以下檔案
				主要檔案: {primary_file}
				附加檔案: {associated_files}
			""", 
			agent=agent, 
			config=config
		)

		await streaming_token(
			message=f"""
				給我剛剛上傳的檔案的metadata和檔案的完整下載連結
			""", 
			agent=agent, 
			config=config
		)

		# await streaming_token(
		# 	message=f"""
		# 		列出主要檔案的所有版本資訊，以及完整的下載連結
		# 	""", 
		# 	agent=agent, 
		# 	config=config
		# )


================================================== Human message ==================================================


				幫我上傳以下檔案
				主要檔案: ./test/test.txt
				附加檔案: ['./test/test.json', './test/test.jpg']
			

================================================== Agent message ==================================================

<think>
Okay, the user wants to upload a primary file and some associated files. Let me check the available functions. There's the upload_file function which takes primary_file, associated_files, archive_ttl, and destroy_ttl. The user provided the main file as ./test/test.txt and associated files as test.json and test.jpg. The parameters are all there. I should call upload_file with those paths. The default TTLs are 30 days, so unless the user specifies otherwise, I can use the defaults. So the arguments would be primary_file: "./test/test.txt", associated_files: ["./test/test.json", "./test/test.jpg"], and the TTLs as default. That should cover the upload.
</th